# **Optimizing Embedding Indices**
After doing a little research, it seems that `pgvector` requires the use of an index in order to do approximate nearest neighbor search. There are a couple of different choices I have w.r.t. building an index, so I want to ensure that the index built resembles the "real results" (i.e., exact-nearest-neighbor search results) as much as possible. 

This notebook will contain some experiments on that! 

# Setup
The cells below will set up the rest of the notebook.

I'll start by configuring the kernel: 

In [ ]:
# Change the working directory 
%cd ..

# Enable the autoreload extension, which will automatically load in new code as it's written
%load_ext autoreload
%autoreload 2

Now I'll import some necessary modules:

In [ ]:
# General import statements
import pandas as pd
import time
from pandas_gbq import read_gbq
from sqlalchemy import create_engine, MetaData
from sqlalchemy.orm import sessionmaker, declarative_base
from tqdm import tqdm
from pathlib import Path
from google.cloud import storage
import json
from concurrent.futures import ThreadPoolExecutor, as_completed
from scipy.stats import kendalltau
import plotly.express as px

# Importing modules custom-built for this project
from utils.settings import (
    GBQ_PROJECT_ID,
    GBQ_DATASET_ID,
    POSTGRES_USER,
    POSTGRES_PASSWORD,
    POSTGRES_HOST,
    POSTGRES_PORT,
    POSTGRES_DB,
    LOG_TO_CONSOLE,
)
from utils.logging import get_logger
from utils.postgres import query_postgres, upload_to_table
from utils.postgres_queries import most_similar_embeddings_to_text_filtered
from utils.search import neural_search

# Set up a logger for this notebook
logger = get_logger("postgres_notebook", log_to_console=LOG_TO_CONSOLE)

Next up: we're going to set up the Postgres engine via SQLAlchemy!

In [ ]:
# Create the connection string to the database
postgres_connection_string = f"postgresql://{POSTGRES_USER}:{POSTGRES_PASSWORD}@{POSTGRES_HOST}:{POSTGRES_PORT}/{POSTGRES_DB}"

# Create the connection engine
engine = create_engine(postgres_connection_string)
metadata = MetaData()
session = sessionmaker(bind=engine)()
Base = declarative_base()

And, finally: below, I'll declare a couple of queries for testing the results:

In [ ]:
# Define some different queries to test throughout the notebook
queries_to_test = [
    "fear and uncertainty of the future",
    "catchy rhythmic synths, addictive shredding electric guitar",
    "a sense of longing and nostalgia for the past",
    "math rock glitch synths amidst an upbeat rhythm",
]

# Exact Neighbor Search
Before I create any indices, I want to test out the "exact neighbor search". 

In [ ]:
# Set the SET max_parallel_workers_per_gather = 4;
query_postgres("SET max_parallel_workers_per_gather = 6;", engine=engine)

# Drop the index if it already exists
query_postgres(
    "DROP INDEX IF EXISTS embeddings_embedding_idx;",
    engine=engine,
    logger=logger,
)

# We're going to store the results of the queries in this list, and then concatenate them into a single DataFrame at the end
exact_search_query_result_dfs = []

# Iterate through each of the queries and run them in the database
for query in tqdm(queries_to_test):

    # Use the neural search for the current query
    start_time = time.time()
    cur_query_results = neural_search(
        query=query,
        n_videos_to_return=10,
        nearest_neighbors_to_screen=None,
    )
    query_time = time.time() - start_time

    # Load the results into a DataFrame
    cur_query_results_df = pd.DataFrame.from_records(json.loads(cur_query_results))

    # Add the query to the DataFrame
    cur_query_results_df["query"] = query

    # Add the query time to the DataFrame
    cur_query_results_df["query_time"] = query_time

    # Append the results to the list
    exact_search_query_result_dfs.append(cur_query_results_df)

# Finally, concatenate the results into a single DataFrame
exact_search_query_results_df = pd.concat(exact_search_query_result_dfs)

# Approximate Nearest Neighbor Search with Indices
Next up, I want to try to create a couple of different indices. For each index, I'm going to try to run the "post-filtering" approach using different amounts of `nearest_neighbors_to_screen`. The goal here: understand which (`index_type`, `index_build_parameters`, `nearest_neighbors_to_screen`) configurations get me closest to the results I got for the exact neighbor search. 

I'll start by generating a query that'll build an index with a particular configuration:

In [ ]:
def create_embeddings_index(
    index_type: str, index_build_parameters: dict, distance_metric: str
):
    """
    This method will create an index on the embeddings table in the Postgres database.
    """

    # First: drop the index if it already exists
    query_postgres(
        "DROP INDEX IF EXISTS embeddings_embedding_idx;",
        engine=engine,
        logger=logger,
    )

    # Unpack the index_build_parameters into a tuple
    index_build_param_tuple = (
        "(" + ", ".join([f"{k} = {v}" for k, v in index_build_parameters.items()]) + ")"
    )

    # Build the index creation query if the index type is "hnsw"
    if index_type == "hnsw":

        # Create the index creation query
        index_creation_str = f"CREATE INDEX ON embeddings USING hnsw (embedding {distance_metric}) WITH {index_build_param_tuple};"

    # Build the index creation query if the index type is "ivfflat"
    elif index_type == "ivfflat":

        # Create the index creation query
        index_creation_str = f"CREATE INDEX ON embeddings USING ivfflat (embedding {distance_metric}) WITH {index_build_param_tuple};"

    # Now, we're going to create the embeddings index
    print(f"RUNNING THE FOLLOWING QUERY: \n{index_creation_str}\n")
    query_postgres(
        index_creation_str,
        engine=engine,
        logger=logger,
    )

Now, with this method in hand, I'm going to define a couple of configurations I'm interested in trying: 

In [ ]:
# Define some of the configurations to try
configurations_to_try = {
    "ivfflat@lists=10": ("ivfflat", {"lists": 10}),
    "ivfflat@lists=50": ("ivfflat", {"lists": 50}),
    "ivfflat@lists=100": ("ivfflat", {"lists": 100}),
}

# Define the different values of nearest_neighbors_to_screen to try
nearest_neighbor_screening_vals = [1000, 10000, 50000]
probes_vals = [1, 4, 10]

Finally: we'll run through each configuration, create the index, and then save the results:

In [ ]:
# We're going to store all of the resulting DataFrames in this list
approximate_search_query_result_dfs = []
query_time_results = {}
index_build_time_results = {}

# Iterate through each of the configurations in the configurations_to_try dictionary
for config_name, config_tuple in configurations_to_try.items():

    query_postgres(
        "SET max_parallel_maintenance_workers = 7; -- plus leader",
        engine=engine,
        logger=logger,
    )
    query_postgres("SET maintenance_work_mem = '6GB';", engine=engine, logger=logger)

    # Run the create_embeddings_index method
    index_build_time_start = time.time()
    create_embeddings_index(
        index_type=config_tuple[0],
        index_build_parameters=config_tuple[1],
        distance_metric="vector_ip_ops",
    )

    # Store the time it took to build the index
    index_build_time_results[config_name] = time.time() - index_build_time_start
    print(
        f"Index build time for {config_name}: {index_build_time_results[config_name]:.2f} seconds"
    )

    # Iterate through each of the different values of probes
    cur_config_probes_list = list(probes_vals)

    # If the current configuration is not ivfflat, set the probes list to [1]
    if config_tuple[0] != "ivfflat":
        cur_config_probes_list = [1]

    for cur_probes_val in probes_vals:

        print(f"\n\nRUNNING WITH PROBES = {cur_probes_val}\n\n")

        # If the current probes val is larger than the number of lists, skip it
        if cur_probes_val > config_tuple[1].get("lists", 0):
            continue

        # Set the current probes value in the index
        query_postgres(
            f"SET ivfflat.probes = {cur_probes_val};",
            engine=engine,
            logger=logger,
        )

        # Iterate through each of the different values of nearest_neighbors_to_screen
        for cur_nearest_neighbor_val in nearest_neighbor_screening_vals:

            # Iterate through each of the queries and run them in the database
            for query in tqdm(queries_to_test):

                # Print some information about the current query
                print(
                    f"Running query: '{query}' with nearest_neighbors_to_screen={cur_nearest_neighbor_val} and configuration={config_name}"
                )

                # Time the query
                start_time = time.time()

                # Use the neural search for the current query
                cur_query_results = neural_search(
                    query=query,
                    n_videos_to_return=10,
                    nearest_neighbors_to_screen=cur_nearest_neighbor_val,
                )
                query_total_time = time.time() - start_time

                # Load the results into a DataFrame
                cur_query_results_df = pd.DataFrame.from_records(
                    json.loads(cur_query_results)
                )

                # Add the query to the DataFrame
                cur_query_results_df["query"] = query

                # Add the nearest_neighbors_to_screen value to the DataFrame
                cur_query_results_df["nearest_neighbors_to_screen"] = (
                    cur_nearest_neighbor_val
                )

                # Add the configuration name to the DataFrame
                cur_query_results_df["configuration"] = (
                    f"{config_name}_probes={cur_probes_val}"
                )

                # Add the query time to the DataFrame
                cur_query_results_df["query_time"] = query_total_time

                # Add the probes value to the DataFrame
                cur_query_results_df["probes"] = cur_probes_val

                # Append the results to the list
                approximate_search_query_result_dfs.append(cur_query_results_df)

# Now, we're going to concatenate the results into a single DataFrame
approximate_search_query_results_df = pd.concat(approximate_search_query_result_dfs)

# **Comparing Results**
Now that I've got some exact and nearest-neighbor results, I can compare them. The cell below will determine - for every configuration & `nearest_neighbors_to_screen` value - how many results overlapped with the actual nearest neighbors.

In [ ]:
# Parameterize the results comparison
n_top_results_to_compare = 10

# We're going to collect records about each query's comparison to the exact search results
query_comparison_results_records = []

# Iterate through each query, and compare the results of the exact search to the approximate search
for query in queries_to_test:

    # Get the exact search results for the current query
    exact_search_results = exact_search_query_results_df[
        exact_search_query_results_df["query"] == query
    ]

    # Get a ranked list of the exact search results
    exact_results_ranked_list = list(exact_search_results["url"])[
        :n_top_results_to_compare
    ]

    # Get the approximate_search_query_results_df for the current query
    approximate_search_results = approximate_search_query_results_df[
        approximate_search_query_results_df["query"] == query
    ]

    # Iterate through each of the configurations in the approximate search results
    for config_name in approximate_search_results["configuration"].unique():

        # Get the approximate search results for the current configuration
        cur_config_approximate_search_results = approximate_search_results[
            approximate_search_results["configuration"] == config_name
        ]

        # Iterate through each of the nearest_neighbors_to_screen values
        for nearest_neighbors_to_screen in cur_config_approximate_search_results[
            "nearest_neighbors_to_screen"
        ].unique():

            # Get the approximate search results for the current nearest_neighbors_to_screen value
            cur_nearest_neighbors_to_screen_results = (
                cur_config_approximate_search_results[
                    cur_config_approximate_search_results["nearest_neighbors_to_screen"]
                    == nearest_neighbors_to_screen
                ]
            )

            # Get the ranked list of approximate search results
            cur_approx_config_ranked_list = list(
                cur_nearest_neighbors_to_screen_results["url"]
            )[:n_top_results_to_compare]

            # Run the Kendall Tau test
            tau, p_value = kendalltau(
                exact_results_ranked_list, cur_approx_config_ranked_list
            )

            # Store this information
            query_comparison_results_records.append(
                {
                    "query": query,
                    "configuration": config_name,
                    "nearest_neighbors_to_screen": nearest_neighbors_to_screen,
                    "kendall_tau": tau,
                    "p_value": p_value,
                    "n_overlap": len(
                        set(exact_results_ranked_list).intersection(
                            set(cur_approx_config_ranked_list)
                        )
                    ),
                    "query_time": cur_nearest_neighbors_to_screen_results[
                        "query_time"
                    ].max(),
                    "probes": cur_nearest_neighbors_to_screen_results["probes"].max(),
                }
            )

# Finally, we're going to store the results of the comparison in a DataFrame
query_comparison_results_df = pd.DataFrame.from_records(
    query_comparison_results_records
)

Now that I've got that, I want to understand: how does the configuration change the `n_overlap`? I'll make a box plot below:

In [ ]:
px.box(
    query_comparison_results_df,
    x="configuration",
    y="n_overlap",
    color="configuration",
    title="Number of Overlapping Results",
)

In general: this seems to lead me to believe that the IVFFlat algorithm tends to do a bit better job at actually identifying similar results to the exact nearest neighbor search.

Let's make the same box plot, but analyzing the Kendall's Tau instead:

In [ ]:
px.box(
    query_comparison_results_df,
    x="configuration",
    y="kendall_tau",
    color="configuration",
    title="Kendall Tau",
)

Let's try and plot the effect that the `nearest_neighbors_to_screen` has on the Kendall's Tau:

In [ ]:
configurations_to_include = query_comparison_results_df["configuration"].unique()

n_neighbors_effect_on_kendall_tau_df = (
    (
        query_comparison_results_df[
            query_comparison_results_df["configuration"].isin(configurations_to_include)
        ]
    )
    .groupby(["configuration", "nearest_neighbors_to_screen"])
    .agg({"kendall_tau": "mean"})
    .reset_index()
    .sort_values("nearest_neighbors_to_screen", ascending=True)
)

# Use Plotly to create a line plot of the Kendall's Tau by nearest_neighbors_to_screen
px.line(
    n_neighbors_effect_on_kendall_tau_df,
    x="nearest_neighbors_to_screen",
    y="kendall_tau",
    color="configuration",
    title="Kendall Tau by nearest_neighbors_to_screen",
)

In [ ]:
query_comparison_results_df[query_comparison_results_df["configuration"].str.startswith("ivfflat@lists=10_")].sort_values(
    "n_overlap", ascending=False
).head(20)